<a href="https://colab.research.google.com/github/Ryo-Hatakeyama/test/blob/main/acft_class_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
w = !apt install tree

In [ ]:
from google.colab import files
files.upload()

Saving imagefiles_224_v2.zip to imagefiles_224_v2.zip


In [ ]:
w = !unzip -o imagefiles_224_v2.zip

In [ ]:
# モジュールのインポート
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection
import matplotlib.pyplot as plt
from re import X
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.applications import VGG16,MobileNetV2, ResNet50V2, EfficientNetV2B3
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img

In [ ]:
# パラメーターの初期化
classes = ["A320", "A330", "A350", "A380", "ATR", "B737", "B747", "B767", "B777", "B787", "DHC8"]
num_classes = len(classes)
image_size = 224

In [ ]:
# データの読み込み
X_train, X_test, y_train, y_test = np.load("./imagefiles_224_v2.npy",allow_pickle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
X_train = X_train.astype("float") / 255.0
X_test = X_test.astype("float") /255.0

In [ ]:
#学習用のImageDataGeneratorクラスの作成
augmentation_train_datagen = ImageDataGenerator(
    #左右反転
    horizontal_flip = True,
    #上下平行移動
    height_shift_range = 0.2,
    #左右平行移動
    width_shift_range = 0.2,
    #明るさ調整
    # brightness_range=[0.5,1.5],
)
#学習用のバッチの生成
augmentation_train_generator = augmentation_train_datagen.flow(X_train, y_train, batch_size=32, seed=0)

In [ ]:
# モデルの定義
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# base_model = EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# print('Model loaded')
# model.summary()

x = base_model.output
x = Flatten(input_shape=base_model.output_shape[1:])(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

# model.summary()
for layer in model.layers[:15]:
    layer.trainable = False

In [ ]:
opt = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model.fit(augmentation_train_generator, batch_size=32, epochs=30)

score = model.evaluate(X_test, y_test, batch_size=32)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
27/27 [==============================] - 9s 302ms/step - loss: 2.5023 - accuracy: 0.1091
Epoch 2/30
27/27 [==============================] - 8s 301ms/step - loss: 2.2003 - accuracy: 0.2110
Epoch 3/30
27/27 [==============================] - 8s 308ms/step - loss: 1.9536 - accuracy: 0.3417
Epoch 4/30
27/27 [==============================] - 8s 295ms/step - loss: 1.7546 - accuracy: 0.3981
Epoch 5/30
27/27 [==============================] - 8s 302ms/step - loss: 1.4485 - accuracy: 0.5168
Epoch 6/30
27/27 [==============================] - 8s 306ms/step - loss: 1.2382 - accuracy: 0.5923
Epoch 7/30
27/27 [==============================] - 8s 299ms/step - loss: 1.0281 - accuracy: 0.6763
Epoch 8/30
27/27 [==============================] - 8s 299ms/step - loss: 0.8839 - accuracy: 0.6978
Epoch 9/30
27/27 [==============================] - 8s 304ms/step - loss: 0.8332 - accuracy: 0.7410
Epoch 10/30
27/27 [==============================] - 8s 305ms/step - loss: 0.7223 - accuracy: 0.7638

データ減らす前

aspvgg16_epochs35_aug_0.8056

asp_resnet_epochs35_aug_0.7346

asp_effi_epochs40_aug_0.7531

asp_effi_epochs35_aug_0.7469

224_vgg16_epochs30_aug_0.811

224_vgg16_epochs27_aug_0.8025

224_vgg16_epochs25_aug_0.8210

224_vgg16_epochs24_aug_0.8302

224_vgg16_epochs23_aug_0.8210

224_vgg16_epochs22_aug_0.7870

データ減らした後

224_vgg16_epochs30_aug_0.86

In [ ]:
# keras_model_path = "/content/keras_save"
# model.save(keras_model_path)

In [ ]:
# restored_keras_model = tf.keras.models.load_model(keras_model_path)

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
# !zip -r /content/keras_save.zip /content/keras_save

  adding: content/keras_save/ (stored 0%)
  adding: content/keras_save/assets/ (stored 0%)
  adding: content/keras_save/keras_metadata.pb (deflated 95%)
  adding: content/keras_save/saved_model.pb (deflated 90%)
  adding: content/keras_save/variables/ (stored 0%)
  adding: content/keras_save/variables/variables.index (deflated 70%)
  adding: content/keras_save/variables/variables.data-00000-of-00001 (deflated 10%)


In [ ]:
# 圧縮した zip ファイルをダウンロードする
# files.download("/content/keras_save.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>